In [15]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tensorflow import keras
import tensorflow as tf

BASIC_PATH = "D:/5FH/ML/"

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Define ImageDataGenerators with image preprocessing <br> and  read train, validation and test images

In [2]:
def get_images(path):
    image_train_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.,
                                                             horizontal_flip=True,
                                                             fill_mode="nearest",
                                                             validation_split=0.25
                                                             )
    
    image_test_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.,
                                                             horizontal_flip=True,
                                                             fill_mode="nearest"
                                                             )
   
    
    train_images = image_train_gen.flow_from_directory(
       directory= path + "/train/",
       color_mode="rgb",
       class_mode="binary",
       subset='training',
       target_size=(70, 60),
       seed=42
       )
    
    valid_images = image_train_gen.flow_from_directory(
       directory= path + "/train/",
       color_mode="rgb",
       class_mode="binary",
       subset='validation',
       target_size=(70, 60),
       seed=42
       )
    
    test_images = image_test_gen.flow_from_directory(
       directory= path + "/test/",
       color_mode="rgb",
       class_mode="binary",
       target_size=(70, 60),
       seed=42
       )
    
    return train_images, valid_images, test_images

In [5]:
train_images, valid_images, test_images = get_images(BASIC_PATH +"images/experiment-classifier/clustered-3-improvement/")

Found 63696 images belonging to 2 classes.
Found 21232 images belonging to 2 classes.
Found 42465 images belonging to 2 classes.


## Create the model with 3 hidden layers and one binary output layer

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[70, 60,3]))
model.add(keras.layers.Dense(700, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))  # "sigmoid" activation function for binary classification

In [8]:
model.compile(loss="binary_crossentropy",   ## for binary classification
              optimizer="sgd",    ###  or optimizer=keras.optimizers.SGD(lr=) for defining learnin grate 
              metrics=["accuracy"])

In [9]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("models/Main-Classifier-3-clusters-improved-{epoch:02d}-{val_accuracy:.2f}.hdf5",  monitor='val_accuracy', save_best_only=True)

## Train the model

In [10]:
history = model.fit(train_images, epochs=100, validation_data=valid_images, callbacks=[early_stopping_cb, checkpoint_cb])

Epoch 1/100
1991/1991 [==============================] - 109s 54ms/step - loss: 0.0458 - accuracy: 0.9877 - val_loss: 0.0646 - val_accuracy: 0.9795
Epoch 2/100
1991/1991 [==============================] - 102s 51ms/step - loss: 0.0401 - accuracy: 0.9887 - val_loss: 0.0487 - val_accuracy: 0.9834
Epoch 3/100
1991/1991 [==============================] - 77s 39ms/step - loss: 0.0313 - accuracy: 0.9910 - val_loss: 0.0340 - val_accuracy: 0.9899
Epoch 4/100
1991/1991 [==============================] - 98s 49ms/step - loss: 0.0326 - accuracy: 0.9905 - val_loss: 0.0320 - val_accuracy: 0.9912
Epoch 5/100
1991/1991 [==============================] - 111s 56ms/step - loss: 0.0271 - accuracy: 0.9924 - val_loss: 0.0381 - val_accuracy: 0.9878
Epoch 6/100
1991/1991 [==============================] - 84s 42ms/step - loss: 0.0275 - accuracy: 0.9920 - val_loss: 0.0255 - val_accuracy: 0.9925
Epoch 7/100
1991/1991 [==============================] - 77s 39ms/step - loss: 0.0270 - accuracy: 0.9924 - val_loss

## evaluate the test dataset with the best trained model

In [11]:
model = keras.models.load_model("./models/Main-Classifier-3-clusters-improved-29-1.00.hdf5")

model.evaluate(test_images)
model.summary()

1328/1328 [==============================] - 36s 27ms/step - loss: 0.0118 - accuracy: 0.9974
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 12600)             0         
_________________________________________________________________
dense_40 (Dense)             (None, 700)               8820700   
_________________________________________________________________
dense_41 (Dense)             (None, 100)               70100     
_________________________________________________________________
dense_42 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 11        
Total params: 8,891,821
Trainable params: 8,891,821
Non-trainable params: 0
_________________________________________________________________
